In [1]:
import datasets
import pandas as pd
import transformers
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

import shap

In [2]:
%cd ../../

d:\python\Toxic-comment-classification


In [3]:
path = 'kaggle/input/'
submission_path = 'kaggle/working/transformers/'
comp = 'jigsaw-toxic-comment-classification-challenge/'
clean_data_path = 'clean_data/'
TRAIN_DATA_FILE=f'{path}{comp}train.csv.zip'
TEST_DATA_FILE=f'{path}{comp}test.csv.zip'
CLEAN_TRAIN_DATA_FILE=f'{clean_data_path}data_train_cleaned_vanilla2.txt'
CLEAN_TEST_DATA_FILE=f'{clean_data_path}data_test_cleaned_vanilla2.txt'
SAMPLE_SUBMISSION=f'{path}{comp}sample_submission.csv.zip'
checkpoint_path = 'model_checkpoint/transformers/'

In [4]:
def read_from_file(filename):
    with open(filename, 'r') as f:
        return f.read().splitlines()  

In [5]:
new_df = pd.DataFrame()
train = pd.read_csv(TRAIN_DATA_FILE)
new_df['text'] = read_from_file(CLEAN_TRAIN_DATA_FILE)
new_df['labels'] = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values.tolist()

In [8]:
# Load tokenizer and model from checkpoint

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("results\\checkpoint-17000")
model.config.id2label = {0: 'toxic', 1: 'severe_toxic', 2: 'obscene', 3: 'threat', 4: 'insult', 5: 'identity_hate'}
model.config.label2id = {'toxic': 0, 'severe_toxic': 1, 'obscene': 2, 'threat': 3, 'insult': 4, 'identity_hate': 5}

pred = transformers.pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer,
    device='cuda',
    return_all_scores=True,
)

d:\anaconda\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
d:\anaconda\Lib\site-packages\transformers\pipelines\text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [9]:
sample_row = new_df.sample(n=1, weights=new_df['labels'].apply(lambda x: 1 in x))
sample_row

,text,labels
117581,i should add that this was also done by someon...,"[1, 0, 0, 0, 0, 0]"


In [10]:
explainer = shap.Explainer(pred)
shap_values = explainer(new_df['text'][127783:127784])

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [11]:
shap.plots.text(shap_values)